<a href="https://colab.research.google.com/github/NathanielRose/eeg-294/blob/master/Gumpy-Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Testing for EEG | ML 294**

In [1]:
!pip install gumpy numpy scipy matplotlib

     |████████████████████████████████| 3.0MB 2.9MB/s 
     |████████████████████████████████| 2.2MB 40.2MB/s 
     |████████████████████████████████| 3.2MB 37.3MB/s 
     |████████████████████████████████| 235kB 37.7MB/s 
  Created wheel for pysam: filename=pysam-0.15.3-cp36-cp36m-linux_x86_64.whl size=8791758 sha256=4f33ead33f05e9536dae2bef54ae5738ec5115f67f9a5eab936847dc940b2b56
  Stored in directory: /root/.cache/pip/wheels/85/ab/84/86ca6dda37a6fc85687b67be7345b735cd82f6584bea56f327
Successfully built pysam
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: pluggy 0.7.1
    Uninstalling pluggy-0.7.1:
      Successfully uninstalled pluggy-0.7.1
  Found existing installation: pytest 3.6.4
    Uninstalling pytest-3.6.4:
      Successfully uninstalled pytest-3.6.4


In [0]:
from __future__ import print_function
import os; os.environ["THEANO_FLAGS"] = "device=gpu0"
import os.path
from datetime import datetime
import sys
sys.path.append('../../gumpy')

import gumpy
import numpy as np
import scipy.io
import matplotlib.pyplot as plt

In [25]:
!git clone https://github.com/NathanielRose/eeg-294.git

Cloning into 'eeg-294'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 53 (delta 16), reused 41 (delta 7), pack-reused 0
Unpacking objects: 100% (53/53), done.


In [26]:
sys.path.append('eeg-294/gumpy-deeplearning')
import models

Using TensorFlow backend.


In [29]:
sys.path.append('eeg-294/gumpy-deeplearning/models')
import utils
utils.print_version_info()

2019/10/30
Keras version: 2.2.5


AttributeError: ignored